# One Month Death

In [ ]:
import pickle
file = open("pickle/1MD/1MD_imputation.pkl",'rb')
dataset = pickle.load(file)

In [ ]:
# ----------  init features selection----------
from ProjectML.monthDeath.pre_processing import *
from ProjectML.monthDeath.feature_extraction import *
X, y, dataset = extract_feature(dataset)
X = drop_corr_feature(X, 0.6)
X = best_eight_features(X)
# ----------  end features selection----------


In [ ]:
# load model
file = open("pickle/1MD/model_xgb.pkl",'rb')
xgb = pickle.load(file)

In [ ]:
# evaluation
from ProjectML.general_util.evaluation import *
print("Cross validation cv = 5 ")
balanced_accuracy_score = get_balanced_accuracy(X, y, xgb)
print("Balanced accuracy: %0.2f (+/- %0.2f)" % (balanced_accuracy_score.mean(), balanced_accuracy_score.std() * 2))
f1_score = get_f1_scores(X, y, xgb)
print("f1_score: %0.2f (+/- %0.2f)" % (f1_score.mean(), f1_score.std() * 2))

# Procedural Success

In [ ]:
file2 = open("pickle/PS/PS_imputation.pkl",'rb')
dataset = pickle.load(file2)

In [ ]:
# ----------  load datasets ----------
from ProjectML.proceduralSuccess.pre_processing import *
from ProjectML.proceduralSuccess.feature_extraction import *
X, y, dataset = extract_feature(dataset)
mask, gb_coefs, gb_mask, rf_coefs, rf_mask = voting_feature_selection(X, y)
X = X.loc[:,mask]

X_scaled = pd.read_pickle('pickle/PS/PS_scaled_features.pkl')
X_scaled = X_scaled.loc[:, ['CenterID','PatientID']]


## Random Forest

In [ ]:
# load model
file2 = open("pickle/PS/model_random_forest.pkl",'rb')
rf= pickle.load(file2)

In [ ]:
# ---------- init split test ----------
X_train_val, X_test, y_train_val, y_test = my_l_split(X, y, 0.1)
X_train_val_scal, X_test_scal, y_train_val_scal, y_test_scal = my_l_split(X_scaled, y, 0.1)
# ---------- end split test ----------


# ---------- init split train validation ----------
X_train, X_val, y_train, y_val = my_l_split(X_train_val, y_train_val, 2 / 9)
X_train_scal, X_val_scal, y_train_scal, y_val_scal = my_l_split(X_train_val_scal, y_train_val_scal, 2 / 9)
# ---------- end split train validation ----------


In [ ]:
#evaluation
from ProjectML.general_util.evaluation import *
print("Classification report on test set:")
y_pred = rf.predict(X_test)
rep_test = classification_report(y_test, y_pred)
print(rep_test)

In [ ]:
#cross validation f1 score
clf = RandomForestClassifier(n_estimators=500, criterion='gini', max_features=5, n_jobs=-1, \
                             class_weight='balanced',random_state=SEED)
print("Cross validation cv = 10 ")
f1_mean0, f1_std0, f1_mean1, f1_std1 = my_cross_f1(X, y, clf, cv=10)
print("f1_score: %0.2f (+/- %0.2f) (class 0)" % (f1_mean0, f1_std0 * 1.95))
print("f1_score: %0.2f (+/- %0.2f) (class 1)" % (f1_mean1, f1_std1 * 1.95))

## SVM

In [ ]:
# load model
file2 = open("pickle/PS/model_svm.pkl",'rb')
svm= pickle.load(file2)

In [ ]:
from ProjectML.general_util.evaluation import *
print("Classification report on test set:")
y_pred = svm.predict(X_test_scal)
rep_test = classification_report(y_test_scal, y_pred)
print(rep_test)

In [ ]:
#cross validation f1 score
clf = svm.SVC(C=100.0, kernel='rbf', class_weight='balanced', max_iter=-1, random_state=SEED)
print("SVM: cross validation cv = 10 ")
f1_mean0, f1_std0, f1_mean1, f1_std1 = my_cross_f1(X_scaled, y, clf, cv=10)
print("f1_score: %0.2f (+/- %0.2f) (class 0)" % (f1_mean0, f1_std0 * 1.95))
print("f1_score: %0.2f (+/- %0.2f) (class 1)" % (f1_mean1, f1_std1 * 1.95))